<h1 style="text-align: center;" markdown="1"> Simple Story Generator </h1>

In [7]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time

In [44]:
# Target log path
logs_path = '/tmp/tensorflow/rnn_words'
writer = tf.summary.FileWriter(logs_path)

# Text file containing words for training
training_file ='belling_the_cat.txt'

#### Apply Preprocesing Learned 

In [45]:
with open(training_file) as f:
    content = f.readlines()

In [23]:
#### apply preprocess...

* **strip()** returns a copy of the string in which all chars have been stripped from the beginning and the end of the string

* **split()** returns a list of all the words in the string, using str as the separator (splits on all whitespace if left unspecified), optionally limiting the number of splits to num.

In [24]:
# strip() removes white spaced and \n at beggining and end
content = [x.strip() for x in content]
print(content)

['long ago , the mice had a general council to consider what measures they could take to outwit their common enemy , the cat . some said this , and some said that but at last a young mouse got up and said he had a proposal to make , which he thought would meet the case . you will all agree , said he , that our chief danger consists in the sly and treacherous manner in which the enemy approaches us . now , if we could receive some signal of her approach , we could easily escape from her . i venture , therefore , to propose that a small bell be procured , and attached by a ribbon round the neck of the cat . by this means we should always know when she was about , and could easily retire while she was in the neighbourhood . this proposal met with general applause , until an old mouse got up and said  that is all very well , but who is to bell the cat ? the mice looked at one another and nobody spoke . then the old mouse said it is easy to propose impossible remedies .']


In [25]:
# split content token by token (it returns a list of tokens)
content = [word for word in content[0].split()]

training_data = np.array(content)
print(training_data)
print("Loaded training data...")

['long' 'ago' ',' 'the' 'mice' 'had' 'a' 'general' 'council' 'to'
 'consider' 'what' 'measures' 'they' 'could' 'take' 'to' 'outwit' 'their'
 'common' 'enemy' ',' 'the' 'cat' '.' 'some' 'said' 'this' ',' 'and'
 'some' 'said' 'that' 'but' 'at' 'last' 'a' 'young' 'mouse' 'got' 'up'
 'and' 'said' 'he' 'had' 'a' 'proposal' 'to' 'make' ',' 'which' 'he'
 'thought' 'would' 'meet' 'the' 'case' '.' 'you' 'will' 'all' 'agree' ','
 'said' 'he' ',' 'that' 'our' 'chief' 'danger' 'consists' 'in' 'the' 'sly'
 'and' 'treacherous' 'manner' 'in' 'which' 'the' 'enemy' 'approaches' 'us'
 '.' 'now' ',' 'if' 'we' 'could' 'receive' 'some' 'signal' 'of' 'her'
 'approach' ',' 'we' 'could' 'easily' 'escape' 'from' 'her' '.' 'i'
 'venture' ',' 'therefore' ',' 'to' 'propose' 'that' 'a' 'small' 'bell'
 'be' 'procured' ',' 'and' 'attached' 'by' 'a' 'ribbon' 'round' 'the'
 'neck' 'of' 'the' 'cat' '.' 'by' 'this' 'means' 'we' 'should' 'always'
 'know' 'when' 'she' 'was' 'about' ',' 'and' 'could' 'easily' 'retire'
 'wh

### Build the vocabulary

Each word present in the text will be assigned to a number. 

In [27]:
# counts from most popular to less popular
count = collections.Counter(training_data).most_common()

dictionary = dict()
for word, _ in count:
    dictionary[word] = len(dictionary)
reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))

vocab_size = len(dictionary)

print(reverse_dictionary)

{0: ',', 1: 'the', 2: '.', 3: 'and', 4: 'to', 5: 'said', 6: 'a', 7: 'could', 8: 'that', 9: 'cat', 10: 'some', 11: 'this', 12: 'mouse', 13: 'he', 14: 'in', 15: 'we', 16: 'is', 17: 'mice', 18: 'had', 19: 'general', 20: 'enemy', 21: 'but', 22: 'at', 23: 'got', 24: 'up', 25: 'proposal', 26: 'which', 27: 'all', 28: 'of', 29: 'her', 30: 'easily', 31: 'propose', 32: 'bell', 33: 'by', 34: 'she', 35: 'was', 36: 'old', 37: 'long', 38: 'ago', 39: 'council', 40: 'consider', 41: 'what', 42: 'measures', 43: 'they', 44: 'take', 45: 'outwit', 46: 'their', 47: 'common', 48: 'last', 49: 'young', 50: 'make', 51: 'thought', 52: 'would', 53: 'meet', 54: 'case', 55: 'you', 56: 'will', 57: 'agree', 58: 'our', 59: 'chief', 60: 'danger', 61: 'consists', 62: 'sly', 63: 'treacherous', 64: 'manner', 65: 'approaches', 66: 'us', 67: 'now', 68: 'if', 69: 'receive', 70: 'signal', 71: 'approach', 72: 'escape', 73: 'from', 74: 'i', 75: 'venture', 76: 'therefore', 77: 'small', 78: 'be', 79: 'procured', 80: 'attached', 8

### Build the input sequence 

In [46]:
n_input = 3
offset = random.randint(0, n_input + 1)
end_offset = n_input + 1
print('offset: %d, end_offset: %d' %(offset, end_offset))

offset: 1, end_offset: 4


In [51]:
symbols = [ [str(training_data[i])] for i in range(offset, offset+n_input) ]
print('Words: ', symbols)
symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
print('Words in number: ', symbols_in_keys)

Words:  [['ago'], [','], ['the']]
Words in number:  [[38], [0], [1]]


In [71]:
# Convert to numpy array
symbols_in_keys = np.array(symbols_in_keys)
print('Shape: ', symbols_in_keys.shape)

Shape:  (1, 3, 1)


In [72]:
# Adding one external dimension
symbols_in_keys = np.reshape(symbols_in_keys, [-1, n_input, 1])
print('Tensor reshaped: ', symbols_in_keys)
print('Shape: ', symbols_in_keys.shape)

Tensor reshaped:  [[[38]
  [ 0]
  [ 1]]]
Shape:  (1, 3, 1)


### One Hot Encoding: Set up the label for the input sequence

In [65]:
print('Word: ', str(training_data[offset+n_input]))
print('Word Number: ', dictionary[str(training_data[offset+n_input])])

Word:  mice
Word Number:  17


In [66]:
# Initialize for one hot encoding
symbols_out_onehot = np.zeros([vocab_size], dtype=float)
print(symbols_out_onehot)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [67]:
symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
print(symbols_out_onehot)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [68]:
symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])
print(symbols_out_onehot)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


### Parameters for Training

In [77]:
# Parameters
learning_rate = 0.001
training_iters = 10000
display_step = 500
n_input = 3

# number of units in RNN cell
n_hidden = 128 #512

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

In [78]:
def RNN(x, weights, biases):

    # reshape to [1, n_input]: from [[38], [0], [1]] to [[38,  0,  1]]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x, n_input,1)

    # 2-layer LSTM, each layer has n_hidden units.
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])

    # 1-layer LSTM with n_hidden units but with lower accuracy.
    # Average Accuracy= 90.60% 50k iter
    # Uncomment line below to test but comment out the 2-layer rnn.MultiRNNCell above
    # rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [7]:
pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [8]:
# Launch the graph
sess = tf.Session()
sess.run(init)
step = 0
offset = random.randint(0,n_input+1)
end_offset = n_input + 1
acc_total = 0
loss_total = 0

writer.add_graph(sess.graph)

while step < training_iters:
    # Generate a minibatch. Add some randomness on selection process.
    if offset > (len(training_data) - end_offset):
        offset = random.randint(0, n_input+1)

    symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
    symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

    symbols_out_onehot = np.zeros([vocab_size], dtype=float)
    symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
    symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

    _, acc, loss, onehot_pred = sess.run([optimizer, accuracy, cost, pred], \
                                            feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
    loss_total += loss
    acc_total += acc
    if (step+1) % display_step == 0:
        print("Iter= " + str(step+1) + ", Average Loss= " + \
              "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
              "{:.2f}%".format(100*acc_total/display_step))
        acc_total = 0
        loss_total = 0
        symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
        symbols_out = training_data[offset + n_input]
        symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval(session=sess))]
        print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
    step += 1
    offset += (n_input+1)
print("Optimization Finished!")
print("Run on command line.")
print("\ttensorboard --logdir=%s" % (logs_path))
print("Point your web browser to: http://localhost:6006/")

Iter= 500, Average Loss= 5.011180, Average Accuracy= 2.60%
['old', 'mouse', 'said'] - [it] vs [?]
Iter= 1000, Average Loss= 3.758229, Average Accuracy= 6.80%
['at', 'one', 'another'] - [and] vs [well]
Iter= 1500, Average Loss= 3.018095, Average Accuracy= 12.60%
['?', 'the', 'mice'] - [looked] vs [looked]
Iter= 2000, Average Loss= 2.761146, Average Accuracy= 24.40%
[',', 'but', 'who'] - [is] vs [the]
Iter= 2500, Average Loss= 2.382247, Average Accuracy= 36.00%
['all', 'very', 'well'] - [,] vs [general]
Iter= 3000, Average Loss= 2.391021, Average Accuracy= 35.00%
['an', 'old', 'mouse'] - [got] vs [got]
Iter= 3500, Average Loss= 2.138273, Average Accuracy= 48.40%
['the', 'neighbourhood', '.'] - [this] vs [this]
Iter= 4000, Average Loss= 2.258429, Average Accuracy= 44.60%
['while', 'she', 'was'] - [in] vs [about]
Iter= 4500, Average Loss= 2.250500, Average Accuracy= 46.40%
['she', 'was', 'in'] - [the] vs [the]
Iter= 5000, Average Loss= 1.934501, Average Accuracy= 50.80%
[',', 'and', 'could

## Inference

In [9]:
sentence = 'said he had'
sentence = sentence.strip()
words = sentence.split(' ')
print(words)
symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
print(symbols_in_keys)

['said', 'he', 'had']
[5, 13, 18]


In [15]:
for i in range(32):
    print(symbols_in_keys)
    keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
    onehot_pred = sess.run(pred, feed_dict={x: keys})
    onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval(session=sess))
    sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
    symbols_in_keys = symbols_in_keys[1:]
    symbols_in_keys.append(onehot_pred_index)

[28, 78, 0]
[78, 0, 15]
[0, 15, 7]
[15, 7, 30]
[7, 30, 72]
[30, 72, 73]
[72, 73, 2]
[73, 2, 105]
[2, 105, 0]
[105, 0, 36]
[0, 36, 100]
[36, 100, 16]
[100, 16, 57]
[16, 57, 1]
[57, 1, 40]
[1, 40, 56]
[40, 56, 27]
[56, 27, 7]
[27, 7, 44]
[7, 44, 72]
[44, 72, 47]
[72, 47, 7]
[47, 7, 81]
[7, 81, 12]
[81, 12, 70]
[12, 70, 28]
[70, 28, 78]
[28, 78, 0]
[78, 0, 15]
[0, 15, 7]
[15, 7, 30]
[7, 30, 72]


In [16]:
print(sentence)



said he had last very meet . and that is a proposal escape from and nobody , old who is agree the consider will all could take escape common could ribbon mouse signal of be , we could easily escape from . nobody , old who is agree the consider will all could take escape common could ribbon mouse signal of be , we could easily escape from
